In [1]:
# combine pkl with csv, generate a new dataset with predictions from LSTM
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
data = pd.read_csv('data/2022-08-21_rnne_NC_250_fGScatter_20MeV_KE_01e8_max_trackE_cut_test.csv.xz')

In [2]:
data.head()

,addr.event,addr.run,addr.subRun,anye.hadE,anye.lepE,anye.longestTrackContained,anye.nuE,event.calE,event.charge,event.nHits,...,particle.dir.x,particle.dir.y,particle.dir.z,particle.energy,particle.is_shower,particle.length,particle.nHit,particle.start.x,particle.start.y,particle.start.z
0,99,20000000.0,0,-0.136679,0.192049,1,0.055370,15982000.0,79366.6,224,...,"-0.0967852,0.28877,-0.274801","-0.830813,-0.448184,0.0794038","-0.548071,0.846016,0.958217","0,1,1","1,0,0","163.862,28.4202,9.53754","360,141,47","336.961,337.3,336.796","266.794,266.143,266.867","1106.86,1107.66,1107.4"
1,5,20000000.0,0,-0.087330,0.215172,1,0.127841,11597300.0,57592.0,183,...,"-0.126556,-0.329956,0.800658","0.264708,0.943707,-0.553893","0.955988,-0.0233775,0.228363","1,1,1","0,0,0","40.0882,17.1144,13.6067","217,74,39","-157.186,-178.165,-161.858","534.979,585.416,545.576","454.069,455.621,492.181"
2,93,20000000.0,0,-0.645356,0.523259,1,-0.122097,45601500.0,226457.0,821,...,"0.27275,-0.118174,-0.200137,0.37859,-0.519619,...","-0.230941,0.203086,0.058152,0.230767,-0.633998...","0.933956,0.972004,0.978041,-0.896335,0.57275,-...","1,1,1,1,1,1,0,1,1,1,0,1","0,0,0,0,0,0,1,0,0,0,1,0","180.179,60.1006,33.6844,15.8725,21.8045,10.694...","887,303,206,99,95,91,94,68,41,30,25,36","-200.123,-203.736,-200.366,-191.685,-203.903,-...","128.288,127.822,129.806,149.609,131.111,128.43...","1133.65,1122.94,1134.25,1114.94,1175.32,1132.9..."
3,78,20000000.0,0,-0.578514,0.401140,1,-0.177374,43901700.0,218016.0,791,...,"0.356433,0.767103,0.47096,-0.355284,-0.515827,...","0.299392,-0.618777,0.85347,-0.136909,0.493977,...","0.885054,0.169314,-0.223126,0.924678,0.699935,...","1,1,0,1,1,1,1,1","0,0,1,0,0,0,0,0","126.086,103.43,173.651,33.1497,16.9031,25.9381...","679,339,360,186,64,71,77,26","-316.395,-314.701,-279.73,-316.809,-319.536,-3...","83.3303,81.8249,115.174,83.778,85.7755,80.4193...","1103.96,1103.51,1200.6,1102.68,1106.68,1102.88..."
4,42,20000000.0,0,0.057597,0.159493,1,0.217090,54993900.0,273099.0,956,...,"0.160711,-0.231041,-0.733035","-0.978808,0.0767519,-0.181711","0.126914,0.969912,0.65547","0,1,0","1,0,1","186.032,10.7939,14.169","2088,61,43","-91.2953,-110.434,-113.597","-318.043,-294.839,-293.18","584.17,687.611,697.884"


In [3]:
data['particle.calE']

0                       2.3955e+07,1.32523e+07,5.07754e+06
1                       1.06728e+07,7.0045e+06,7.17576e+06
2        4.09547e+07,1.50999e+07,1.8616e+07,5.43899e+06...
3        3.35255e+07,1.72717e+07,2.43053e+07,1.22182e+0...
4                      1.35137e+08,2.93231e+06,2.10603e+06
                               ...                        
21247       2.10704e+07,4.55553e+06,3.98222e+06,1.7464e+06
21248      1.40396e+08,3.62253e+07,1.08888e+07,2.20312e+06
21249                               2.44262e+07,8.6497e+06
21250    1.63449e+07,1.80128e+07,6.83668e+06,1.50821e+0...
21251                                          1.93641e+07
Name: particle.calE, Length: 21252, dtype: object

In [10]:
filename="data/2022-08-21_rnne_NC_250_fGScatter_20MeV_KE_01e8_max_trackE_cut_pred"
with open(filename+".pkl", "rb") as f:
    pred, true = pickle.load(f)

In [28]:
# verify that the predictions are corresponding to the correct data
np.std(true['total']-data['mc.nuE'])

2.467164847444135e-16

In [30]:
len(true['total'])

21252

In [31]:
len(data)

21252

In [32]:
data.insert(0, 'lstm_EE_pred.total', pred['total'])
data.insert(1, 'lstm_EE_pred.primary', pred['primary'])
data.insert(2, 'lstm_EE_pred.secondary', pred['secondary'])
data.to_csv('data/2022-08-21_rnne_NC_250_fGScatter_20MeV_KE_01e8_max_trackE_cut_test_with_pred.csv.xz', index=False, compression='xz')
